# Python для анализа данных

*Ян Пиле, НИУ ВШЭ*

## Интерактивные визуализации в Plotly v2

In [12]:
# !pip install cufflinks
# !pip install fastparquet
# !pip install pyarrow

Это мой небольшой кавер на:
    
https://nbviewer.jupyter.org/github/WillKoehrsen/Data-Analysis/blob/master/plotly/Plotly%20Whirlwind%20Introduction.ipynb#Introduction:-Plotting-with-Plotly-+-Cufflinks-in-Python

In [1]:
# Standard plotly imports
import plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
# Using plotly + cufflinks in offline mode
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

In [21]:
import os

In [22]:
os.getcwd()

'/Users/ianpile/DPO 2020'

In [3]:
df.shape

(126, 24)

In [7]:
type(df['published_date'][0])

pandas._libs.tslibs.timestamps.Timestamp

In [2]:
import pandas as pd
df = pd.read_parquet('/Users/ianpile/Downloads/Data-Analysis-master/medium/data/medium_data_2019_01_06')
df.head()

claps  days_since_publication  fans  num_responses publication  \
121      2              574.742788     2              0        None   
122     18              567.424835     3              0        None   
123     50              554.804959    19              0        None   
119      0              553.962351     0              0        None   
116      0              549.974700     0              0        None   

         published_date  read_ratio  read_time  reads        started_date  \
121 2017-06-10 14:25:00       41.98          7     68 2017-06-10 14:24:00   
122 2017-06-17 22:02:00       32.93         14     54 2017-06-17 22:02:00   
123 2017-06-30 12:55:00       20.19         42    215 2017-06-30 12:00:00   
119 2017-07-01 09:08:00       35.85          9     19 2017-06-30 18:21:00   
116 2017-07-05 08:51:00        8.77         14      5 2017-07-03 20:18:00   

     ...       type views word_count  claps_per_word editing_days  \
121  ...  published   162       1859        0.001076            0   
122  ...  published   164       3891        0.004626            0   
123  ...  published  1065      12025        0.004158            0   
119  ...  published    53       2533        0.000000            0   
116  ...  published    57       3892        0.000000            1   

     <tag>Education  <tag>Data Science  <tag>Towards Data Science  \
121               0                  0                          0   
122               0                  0                          0   
123               0                  0                          0   
119               0                  0                          0   
116               0                  0                          0   

     <tag>Machine Learning  <tag>Python  
121                      0            0  
122                      0            0  
123                      1            1  
119                      0            0  
116                      0            0  

[5 rows x 24 columns]

Типы графиков, которые можно нарисовать с помощью cufflinks:

   
    scatter, bar, box, spread
    ratio, heatmap, surface
    histogram, bubble, bubble3d
    scatter3d, scattergeo, ohlc
    candle, pie, choropleth

**Распределение одной переменной**

Здесь очевидным решением будет гистограмма

In [5]:
df['claps'].iplot(
    kind='hist',
    bins=30,
    xTitle='claps',
    linecolor='black',
    yTitle='count',
    title='Claps Distribution')

На любой прямоугольник можно навести курсор и увидеть значение

**Гистограмма с процентами**

Точно такая же гистограмма в одно действие превращается в распределение с процентами.

In [6]:
df['claps'].iplot(
    kind='hist',
    bins=30,
    xTitle='claps',
    linecolor='black',
    histnorm='percent',
    yTitle='percentage (%)',
    title='Reads Distribution in Percent')

Также на одну гистограмму можно уложить несколько распределений (впрочем, мы это уже проделывали)

In [8]:
def to_time(dt):
    return dt.hour + dt.minute / 60

In [9]:
df['time_started'] = df['started_date'].apply(to_time)
df['time_published'] = df['published_date'].apply(to_time)

df[['time_started', 'time_published']].iplot(
    kind='hist',
    linecolor='black',
    bins=24,
    histnorm='percent',
    bargap=0.1,
    opacity=0.8,
    barmode='group',
    xTitle='Time of Day',
    yTitle='(%) of Articles',
    title='Time Started and Time Published')

А иногда хочется, чтобы распределении не строились рядом друг с другом, а строились одно поверх другого (с перекрытием). За это отвечает аргумент *barmode* которому надо присвоить значение overlay.

In [10]:
df[['time_published', 'time_started']].iplot(
    kind='hist',
    bins=24,
    linecolor='black',
    opacity=0.6,
    histnorm='percent',
    barmode='overlay',
    xTitle='Time of day',
    yTitle='(%) of articles',
    title='Time Started and Time Published Overlaid')

Столбчатые диаграммы обычно строятся поверх группировки датафрейма по некоторому полю (обычно категориальному). Например давайте посчитаем количество статей в публикации.

In [20]:
df.columns

Index(['claps', 'days_since_publication', 'fans', 'num_responses',
       'publication', 'published_date', 'read_ratio', 'read_time', 'reads',
       'started_date', 'tags', 'text', 'title', 'title_word_count', 'type',
       'views', 'word_count', 'claps_per_word', 'editing_days',
       '<tag>Education', '<tag>Data Science', '<tag>Towards Data Science',
       '<tag>Machine Learning', '<tag>Python', 'time_started',
       'time_published'],
      dtype='object')

In [14]:
df['publication'].unique()

array(['None', 'Towards Data Science', 'Engineering @ Feature Labs',
       'Noteworthy - The Journal Blog', 'The Reality Project'],
      dtype=object)

In [15]:
df.groupby('publication').count()['fans'].iplot(
    kind='bar', yTitle='Number of Articles', linecolor='black', title='Articles by Publication')

А еще можно строить столбчатые диаграммы по нескольким столбцам (Если в них однородная информация - у нас, например, есть флажки того, что публикация содержит некий тег (например, Python))

In [21]:
[c for c in df if '<tag>' in c]

['<tag>Education',
 '<tag>Data Science',
 '<tag>Towards Data Science',
 '<tag>Machine Learning',
 '<tag>Python']

In [22]:
df[[c for c in df if '<tag>' in c]].sum().iplot(
    kind='bar',
    xTitle='Tag',
    yTitle='Number of Articles with Tag',
    title='Frequency of Tags',
    linecolor='black',
    sortbars=True)

**А еще можно распределение пары признаков рядом нарисовать**

Установим в качестве индекса published_date, приведем дату к месяцу и усредним :).

In [24]:
df2 = df[['views', 'reads',
          'published_date']].set_index('published_date').resample('M').mean()
df2.head()

views        reads
published_date                          
2017-06-30       463.666667   112.333333
2017-07-31      5521.333333  1207.166667
2017-08-31      6242.800000   993.700000
2017-09-30      2113.000000   279.000000
2017-10-31              NaN          NaN

In [28]:
df3 = df2.dropna()

In [30]:
df3

views         reads
published_date                            
2017-06-30        463.666667    112.333333
2017-07-31       5521.333333   1207.166667
2017-08-31       6242.800000    993.700000
2017-09-30       2113.000000    279.000000
2017-12-31      60818.750000  15724.000000
2018-01-31      31615.944444   9065.722222
2018-02-28      25463.875000   9933.750000
2018-03-31      69373.500000  18884.000000
2018-04-30      35668.333333  11340.333333
2018-05-31      53369.400000  11298.000000
2018-06-30      50161.200000  11675.800000
2018-07-31      17410.000000   2957.333333
2018-08-31      16622.500000   4423.125000
2018-09-30      11755.333333   2564.833333
2018-10-31      16041.400000   4265.400000
2018-11-30      12921.300000   3223.500000
2018-12-31       3844.578947   1188.736842
2019-01-31        354.333333    110.333333

In [29]:
df3.iplot(
    kind='bar',
    xTitle='Дата',
    yTitle='Среднее',
    title='Среднемесячное кол-во просмотров и прочтений')

Кроме того можно на графике добавить еще одну ось Y

In [31]:
df2 = df[['views', 'read_time',
          'published_date']].set_index('published_date').resample('M').mean()

df2.iplot(
    kind='bar',
    xTitle='Дата',
    secondary_y='read_time',
    secondary_y_title='Среднее время прочтения',
    yTitle='Среднее кол-во просмотров',
    title='Средние за месяц')

Конечно же Plotly умеет рисовать и ящики с усами - куда же без них.

In [32]:
df[['claps', 'fans']].iplot(secondary_y='fans', secondary_y_title='Fans',
    kind='box', yTitle='Claps', title='BoxPlot Claps и Fans')

Можно нарисовать набор ящиков с усами, например, для нескольких категорий.

In [33]:
df2 = df.pivot(
    columns='publication', values='fans')
df2.head()

publication  Engineering @ Feature Labs  None  Noteworthy - The Journal Blog  \
0                                   NaN   NaN                            NaN   
1                                   NaN   NaN                            NaN   
2                                   NaN   0.0                            NaN   
3                                   NaN  12.0                            NaN   
4                                   NaN   3.0                            NaN   

publication  The Reality Project  Towards Data Science  
0                           34.0                   NaN  
1                           29.0                   NaN  
2                            NaN                   NaN  
3                            NaN                   NaN  
4                            NaN                   NaN

Всегда в iplot можно явно передать набор параметров в виде словаря(так у нас больше контроля)

In [34]:
df2.iplot(
        kind='box',
        layout=dict(
            height=600,
            yaxis=dict(title='fans'),
            title='Fans by Publication',
            margin=dict(b=140)))

In [35]:
df[df['read_time'] <= 10].pivot(
    columns='read_time', values='reads').iplot(
        kind='box', colorscale='set2',
        xTitle='Время прочтения',
        yTitle='Кол-во прочтений',
        title='Box Plot of Reads by Reading Time')

Сделаем из куска нашего датафрейма временной ряд. Для этого установим дату в индекс

In [36]:
tds = df[df['publication'] == 'Towards Data Science'].set_index(
    'published_date')

tds.head()

claps  days_since_publication  fans  num_responses  \
published_date                                                            
2017-12-27 11:20:00   4800              374.871062   861             27   
2018-01-06 20:15:00    857              364.499289   112              6   
2018-01-07 20:37:00    186              363.484171    45              1   
2018-01-08 16:58:00    119              362.636216    43              2   
2018-01-09 21:49:00   2000              361.434287   392             12   

                              publication  read_ratio  read_time  reads  \
published_date                                                            
2017-12-27 11:20:00  Towards Data Science       17.68         21  28566   
2018-01-06 20:15:00  Towards Data Science       22.76         17   7207   
2018-01-07 20:37:00  Towards Data Science       28.64         15    775   
2018-01-08 16:58:00  Towards Data Science       31.53         11    740   
2018-01-09 21:49:00  Towards Data Science       23.99         12  25505   

                           started_date  \
published_date                            
2017-12-27 11:20:00 2017-12-26 15:11:00   
2018-01-06 20:15:00 2018-01-03 21:38:00   
2018-01-07 20:37:00 2018-01-07 13:18:00   
2018-01-08 16:58:00 2018-01-02 17:23:00   
2018-01-09 21:49:00 2018-01-09 12:26:00   

                                                                  tags  ...  \
published_date                                                          ...   
2017-12-27 11:20:00  [Machine Learning, Data Science, Random Forest...  ...   
2018-01-06 20:15:00  [Machine Learning, Python, Data Science, Learn...  ...   
2018-01-07 20:37:00     [Data Science, Data, Weight Loss, Programming]  ...   
2018-01-08 16:58:00     [Statistics, Data Science, Book Review, Books]  ...   
2018-01-09 21:49:00     [Machine Learning, Python, Data Science, Data]  ...   

                    word_count claps_per_word  editing_days <tag>Education  \
published_date                                                               
2017-12-27 11:20:00       4494       1.068091             0              0   
2018-01-06 20:15:00       3504       0.244578             2              0   
2018-01-07 20:37:00       3569       0.052115             0              0   
2018-01-08 16:58:00       2817       0.042244             5              0   
2018-01-09 21:49:00       2456       0.814332             0              0   

                     <tag>Data Science  <tag>Towards Data Science  \
published_date                                                      
2017-12-27 11:20:00                  1                          0   
2018-01-06 20:15:00                  1                          0   
2018-01-07 20:37:00                  1                          0   
2018-01-08 16:58:00                  1                          0   
2018-01-09 21:49:00                  1                          0   

                     <tag>Machine Learning  <tag>Python  time_started  \
published_date                                                          
2017-12-27 11:20:00                      1            1     15.183333   
2018-01-06 20:15:00                      1            1     21.633333   
2018-01-07 20:37:00                      0            0     13.300000   
2018-01-08 16:58:00                      0            0     17.383333   
2018-01-09 21:49:00                      1            1     12.433333   

                     time_published  
published_date                       
2017-12-27 11:20:00       11.333333  
2018-01-06 20:15:00       20.250000  
2018-01-07 20:37:00       20.616667  
2018-01-08 16:58:00       16.966667  
2018-01-09 21:49:00       21.816667  

[5 rows x 25 columns]

In [37]:
tds['read_time'].iplot(
    mode='lines+markers',
    opacity=0.8,
    size=8,
    symbol=1,
    xTitle='Дата',
    yTitle='Минимальное время прочтения',
    title='Read Time Trends')

Конечно же на одном графике можно отображать несколько рядов:

In [38]:
tds[['claps', 'fans']].iplot(
    mode='lines+markers',
    opacity=0.8,
    size=8,
    symbol=1,
    xTitle='Date',
    yTitle='Fans and Claps',
    title='Fans and Claps over Time')

А можно явным образом настроить две оси Y.

In [70]:
tds[['fans', 'word_count', 'title']].iplot(
    y='fans',
    mode='lines+markers',
    secondary_y = 'word_count',
    secondary_y_title='Word Count',
    opacity=0.8,
    size=8,
    symbol=1,
    xTitle='Date',
    yTitle='Claps',
    text='title',
    title='Fans and Word Count over Time')

Данные на графиках можно подписывать, но это делается в html-разметке (На мой взгляд, это единственная не слишком удобная часть plotly)

In [44]:
list(zip(tds_monthly_totals.index.month_name(),
                    tds_monthly_totals['word_count']))

[('December', 4494),
 ('January', 29110),
 ('February', 12022),
 ('March', 14064),
 ('April', 11526),
 ('May', 14594),
 ('June', 15224),
 ('July', 9263),
 ('August', 19426),
 ('September', 12922),
 ('October', 10224),
 ('November', 21517),
 ('December', 11176)]

In [45]:
tds_monthly_totals = tds.resample('M').sum()

tds_monthly_totals['text'] = [
    f'<span style="color:blue">{m}<span><br>words: {w:.0f}'
    for m, w in zip(tds_monthly_totals.index.month_name(),
                    tds_monthly_totals['word_count'])
]

tds_monthly_totals.iplot(
    mode='lines+markers+text',
    text='text',
    y='word_count',
    opacity=0.8,
    xTitle='Date',
    yTitle='Word Count',
    title='Total Word Count by Month')

Диаграммы рассеяния мы уже строили, но у них есть еще кое-что, что мы не охватили

In [46]:
tds.iplot(
    x='read_time',
    y='read_ratio',
    xTitle='Read Time',
    yTitle='Reading Percent',
    text='title',
    mode='markers',
    title='Reading Percent vs Reading Time')

In [47]:
tds.sort_values('read_time').iplot(
    x='read_time',
    y='read_ratio',
    xTitle='Read Time',
    yTitle='Reading Percent',
    text='title',
    mode='markers+lines',
    bestfit=True, bestfit_colors=['blue'],
    title='Reading Percent vs Reading Time')

Помимо этого графики могут быть построены "с накоплением"

In [48]:
df.set_index('published_date')[['views', 'word_count']].cumsum().iplot(y='views', secondary_y='word_count',
                                                 yTitle='Views', secondary_y_title='Word Count',
                                                 title='Views and Word Count Totals')

Кроме того, plotly умеет строить парные графики по типу того, что мы делали в seaborn

In [49]:
import plotly.figure_factory as ff

figure = ff.create_scatterplotmatrix(
    df[['claps', 'publication', 'views', 'read_ratio', 'word_count']],
    height=1000,
    width=1000,
    text=df['title'],
    diag='histogram',
    index='publication')
iplot(figure)

/Users/ianpile/opt/anaconda3/lib/python3.7/site-packages/plotly/tools.py:465: DeprecationWarning:

plotly.tools.make_subplots is deprecated, please use plotly.subplots.make_subplots instead



Кроме того можно рисовать график разброса между двумя переменными(Например такое можно провернуть для обменных курсов валют)

In [79]:
df.set_index('published_date')[['views', 'reads']].iplot(
    kind='spread', mode='markers', yTitle='Number', title='Spread between Views and Reads')

/Users/ianpile/opt/anaconda3/lib/python3.7/site-packages/cufflinks/plotlytools.py:850: FutureWarning:

The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead



In [80]:
cufflinks.colors.scales()

Ну и на закуску нарисуем пирожок :)

In [81]:
df.groupby(
    'publication', as_index=False)['reads'].count().iplot(
        kind='pie', labels='publication', values='reads', title='Percentage of Reads by Publication')

И еще один!

In [83]:
df.groupby(
    'publication', as_index=False)['word_count'].sum().iplot(
        kind='pie', labels='publication', values='word_count', title='Percentage of Words by Publication')